In [1]:
import contextlib
import io

import numpy as np
import pandas as pd
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns 

from scipy.interpolate import interp1d

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import LSTM, Bidirectional, GRU

In [ ]:
# Température - Tour Sud - 2ème Étage
tour_sud_2_temp = ['S2STemp', 'S2OTemp', 'S2ETemp', 'S2NTemp']

# Désignation: Tour-Etage-Orientation-Temp/HR (S1NTemp: Tour Sud- 1er Etage- Orientation Nord- Température)
designations = {
    'N1NTemp': 'Tour Nord - 1er Étage - Orientation Nord - Température',
    'N1ETemp': 'Tour Nord - 1er Étage - Orientation Est - Température',
    'N1STemp': 'Tour Nord - 1er Étage - Orientation Sud - Température',
    'N1OTemp': 'Tour Nord - 1er Étage - Orientation Ouest - Température',
    'S1NTemp': 'Tour Sud - 1er Étage - Orientation Nord - Température',
    'S1OTemp': 'Tour Sud - 1er Étage - Orientation Ouest - Température',
    'N2STemp': 'Tour Nord - 2ème Étage - Orientation Sud - Température',
    'N2OTemp': 'Tour Nord - 2ème Étage - Orientation Ouest - Température',
    'N2ETemp': 'Tour Nord - 2ème Étage - Orientation Est - Température',
    'N2NTemp': 'Tour Nord - 2ème Étage - Orientation Nord - Température',
    'S2STemp': 'Tour Sud - 2ème Étage - Orientation Sud - Température',
    'S2OTemp': 'Tour Sud - 2ème Étage - Orientation Ouest - Température',
    'S2ETemp': 'Tour Sud - 2ème Étage - Orientation Est - Température',
    'S2NTemp': 'Tour Sud - 2ème Étage - Orientation Nord - Température',
    'S2OHR': 'Tour Sud - 2ème Étage - Orientation Ouest - Humidité Relative',
    'S2SHR': 'Tour Sud - 2ème Étage - Orientation Sud - Humidité Relative',
    'S2EHR': 'Tour Sud - 2ème Étage - Orientation Est - Humidité Relative',
    'S2NHR': 'Tour Sud - 2ème Étage - Orientation Nord - Humidité Relative',
    'N2SHR': 'Tour Nord - 2ème Étage - Orientation Sud - Humidité Relative',
    'N2OHR': 'Tour Nord - 2ème Étage - Orientation Ouest - Humidité Relative',
    'N2EHR': 'Tour Nord - 2ème Étage - Orientation Est - Humidité Relative',
    'N2NHR': 'Tour Nord - 2ème Étage - Orientation Nord - Humidité Relative',
    'tour_nord_1_temp': 'Température - Tour Nord - 1er Étage',
    'tour_sud_1_temp': 'Température - Tour Sud - 1er Étage',
    'tour_sud_2_temp': 'Température - Tour Sud - 2ème Étage',
    'tour_nord_2_temp': 'Température - Tour Nord - 2ème Étage'
}

In [ ]:
# Températures, Humidité
Temp_HR = ['N1NTemp', 'N1ETemp', 'N1STemp', 'N1OTemp', 'S1NTemp', 'S1OTemp', 'N2STemp', 'N2OTemp', 'N2ETemp', 'N2NTemp', 'S2STemp', 'S2OTemp', 'S2ETemp', 'S2NTemp', 'N2SHR', 'N2OHR', 'N2EHR', 'N2NHR', 'S2SHR', 'S2OHR', 'S2EHR', 'S2NHR']

# Humidité, Températures (Tour Nord 2 Etages)
tour_nord_2_temp_HR = [("N2STemp", "N2SHR"), ("N2OTemp", "N2OHR"), ("N2ETemp", "N2EHR"), ("N2NTemp", "N2NHR")]
tour_sud_2_temp_HR = [("S2STemp", "S2SHR"), ("S2OTemp", "S2OHR"), ("S2ETemp", "S2EHR"), ("S2NTemp", "S2NHR")]

In [ ]:
def split_dataframe(df, train_size=0.7, val_size=0.2):
    """
    Découpe un DataFrame temporel en trois parties : train, val, test,
    en respectant l'ordre chronologique.

    Args:
        df (pd.DataFrame): Données à découper (indexé ou non par le temps)
        train_size (float): Proportion pour l'ensemble d'entraînement
        val_size (float): Proportion pour la validation

    Returns:
        df_train, df_val, df_test (DataFrames)
    """
    n = len(df)
    train_end = int(n * train_size)
    val_end = int(n * (train_size + val_size))

    df_train = df.iloc[:train_end]
    df_val = df.iloc[train_end:val_end]
    df_test = df.iloc[val_end:]

    print(f"Train size : {len(df_train)}")
    print(f"Val size : {len(df_val)}")
    print(f"Test size : {len(df_test)}")

    return df_train.copy(), df_val.copy(), df_test.copy()

In [ ]:
def evaluate_model(y_true, y_pred):
    """
    Calcule et affiche les métriques MAE, RMSE et R² entre les vraies valeurs et les prédictions.

    Args:
        y_true (array-like): Valeurs réelles.
        y_pred (array-like): Valeurs prédites.
    """
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)

    print(f'MAE: {mae:.4f}')
    print(f'RMSE: {rmse:.4f}')
    print(f'R2 Score: {r2:.4f}')
    
    return mae, rmse, r2

In [ ]:
def create_dataset(X, y, time_steps=1):  
    Xs, ys = [], []   
    for i in range(len(X) - time_steps):   
        v = X.iloc[i:(i + time_steps)].values 
        Xs.append(v)      
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

    """
This function prepares the input features and target values in the format required for training a recurrent neural network (RNN) or LSTM model for sequential prediction tasks. It creates sequences of input features and their corresponding target values, which can be fed into the model during training.

    - X: This parameter represents the input features, typically a pandas DataFrame containing multiple time-series variables such as temperature, humidity, etc.
    - y: This parameter represents the target values, which are typically the values we want to predict based on the input features.
    - time_steps: This parameter defines the length of each sequence. It determines how many data points from the past will be used to predict the next data point. For example, if time_steps is set to 3, the function will create sequences of three consecutive data points as input features and the next data point as the target value.
    """

In [ ]:
def build_and_train_bilstm(X_train, y_train, X_val, y_val, sequence_length, units=100, 
                           activation='tanh', epochs=30, batch_size=32, patience=10):
    """
    Construit, entraîne et évalue un modèle BiLSTM avec early stopping.

    Args:
        X_train (ndarray): Données d'entraînement (3D).
        y_train (ndarray): Cibles d'entraînement.
        X_val (ndarray): Données de validation (3D).
        y_val (ndarray): Cibles de validation.
        sequence_length (int): Longueur des séquences en entrée.
        units (int): Nombre de neurones dans la couche LSTM.
        activation (str): Fonction d'activation de la couche LSTM.
        epochs (int): Nombre d’époques d’entraînement.
        batch_size (int): Taille de lot pour l’entraînement.
        patience (int): Patience pour l’early stopping.

    Returns:
        model: Le modèle entraîné.
        history: L'historique d'entraînement.
        val_loss: La perte de validation finale.
    """
    model = Sequential()
    model.add(Bidirectional(LSTM(units, activation=activation, input_shape=(sequence_length, X_train.shape[2]))))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    early_stopping = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early_stopping],
        verbose=1
    )

    val_loss = model.evaluate(X_val, y_val, verbose=0)
    display(f'Validation Loss: {val_loss}')

    return model, history, val_loss

In [ ]:
def load_data_drias(path_fichier_excel):
    """
    Charge un fichier Excel DRIAS avec une colonne 'Date' au format '%d/%m/%Y',
    et retourne un DataFrame avec la date en index.

    Args:
        path_fichier_excel (str): Chemin vers le fichier Excel.

    Returns:
        pd.DataFrame: Données DRIAS avec l'index daté.
    """
    df = pd.read_excel(path_fichier_excel)
    df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
    df.set_index('Date', inplace=True)
    display(df)
    return df

In [ ]:
def predict_and_inverse_transform(model, X, y, target_transformer):
    """
    Effectue la prédiction avec le modèle donné et applique l'inverse de la transformation 
    sur les prédictions et les vraies valeurs cibles.

    Args:
        model: Modèle entraîné (ex. BiLSTM).
        X_val: Données d'entrée de validation.
        y_val: Vraies valeurs cibles de validation.
        target_transformer: Transformateur utilisé pour normaliser les cibles (ex. MinMaxScaler).

    Returns:
        Tuple (y_pred_original_scale, y_true_original_scale)
    """
    # Prédiction
    y_pred = model.predict(X)

    # Inversion de la transformation des prédictions
    y_pred_inv = target_transformer.inverse_transform(y_pred)

    # Reshape puis inversion de la transformation des vraies valeurs
    y = y.reshape(-1, 1)
    y_val_inv = target_transformer.inverse_transform(y)

    return y_pred_inv, y_val_inv

In [ ]:
def temp_mean_max_min(data1, data2, label1, label2, start_year=2006, end_year=2025, plot_min_max=False):
    """
    Compare les températures (moyenne, min, max) de deux ensembles de données 
    sur une série d'années données en superposant les courbes de chaque année
    sur un axe temporel standardisé (année 2000).

    Paramètres :
        data1, data2 : DataFrames pandas avec un index datetime.
        label1, label2 : noms à afficher pour identifier les deux jeux de données.
        start_year, end_year : année de début et de fin (incluses).
        plot_min_max : booléen pour activer/désactiver les courbes min et max.
    """
    annees = range(start_year, end_year + 1)
    n_subplots = len(annees)
    n_cols = 2
    n_rows = n_subplots

    plt.figure(figsize=(n_cols * 14, n_rows * 5))
    
    for i, annee in enumerate(annees, start=1):
        # ----- Sous-graphique 1 : data1 -----
        plt.subplot(n_rows, n_cols, 2 * (i - 1) + 1)
        df1 = data1[data1.index.year == annee].copy()
        df1.index = df1.index.map(lambda d: d.replace(year=2000))

        df1["tasAdjust"].plot(label=f"Temp Moyenne {annee}", color="blue", lw=2)
        if plot_min_max:
            df1.get("tasmaxAdjust", pd.Series(index=df1.index)).plot(label="Max", color="red", lw=1)
            df1.get("tasminAdjust", pd.Series(index=df1.index)).plot(label="Min", color="green", lw=1)

        _format_axes(title=f"{label1} : {annee}")

        # ----- Sous-graphique 2 : data2 -----
        plt.subplot(n_rows, n_cols, 2 * (i - 1) + 2)
        df2 = data2[data2.index.year == annee].copy()
        df2.index = df2.index.map(lambda d: d.replace(year=2000))

        df2["2m_Temperature_C"].plot(label=f"Temp Moyenne {annee}", color="blue", lw=2)
        if plot_min_max:
            df2.get("MAX_TEMPERATURE_C", pd.Series(index=df2.index)).plot(label="Max", color="red", lw=1)
            df2.get("MIN_TEMPERATURE_C", pd.Series(index=df2.index)).plot(label="Min", color="green", lw=1)

        _format_axes(title=f"{label2} : {annee}")

    plt.tight_layout()
    plt.show()

def _format_axes(title):
    """Formate les axes d'une sous-figure avec date sur l'axe X"""
    ax = plt.gca()
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
    plt.xlabel("Mois")
    plt.ylabel("Température (°C)")
    plt.title(title)
    plt.ylim(-10, 40)
    plt.grid(True)
    plt.legend()


In [ ]:
def quantile_mapping_correction(obs_series, mod_series, n_quantiles=100):
    """
    Correction de biais par méthode Quantile Mapping entre séries d'observations et de modèles

    Paramètres :
        obs_series : pd.Series (Données observées, index temporel)
        mod_series : pd.Series (Données du modèle à corriger, index temporel)
        n_quantiles : int (Nombre de quantiles pour la correction)

    Retour :
        pd.Series (Série corrigée du modèle)
    """
    # Vérification des entrées
    if not isinstance(obs_series, pd.Series) or not isinstance(mod_series, pd.Series):
        raise TypeError("Les entrées doivent être des pandas Series")
    
    # Trouver l'intersection temporelle
    common_index = obs_series.index.intersection(mod_series.index)
    
    if len(common_index) == 0:
        raise ValueError("Aucune période d'intersection entre les séries")
    
    # Échantillonnage des données communes
    obs_common = obs_series.loc[common_index].dropna()
    mod_common = mod_series.loc[common_index].dropna()
    
    # Vérification des points de données
    n_points = min(len(obs_common), len(mod_common))
    if n_points < 2:
        raise ValueError("Pas assez de données communes pour la correction")
    
    # Calcul des quantiles (avec protection contre le surapprentissage)
    n_quantiles = min(n_quantiles, max(n_points // 10, 2))
    quantiles = np.linspace(0, 1, n_quantiles)
    
    # Calcul des valeurs de quantiles
    obs_quantiles = np.nanquantile(obs_common, quantiles)
    mod_quantiles = np.nanquantile(mod_common, quantiles)
    
    # Création des fonctions d'interpolation
    # CDF du modèle
    cdf_model = interp1d(
        mod_quantiles, 
        quantiles, 
        kind='linear',
        bounds_error=False, 
        fill_value=(0, 1)
    )  # Parenthèse fermante ajoutée ici
    
    # Inverse de la CDF des observations
    inv_cdf_obs = interp1d(
        quantiles, 
        obs_quantiles, 
        kind='linear',
        bounds_error=False,
        fill_value=(obs_quantiles[0], obs_quantiles[-1])
    )  # Parenthèse fermante ajoutée ici
    
    # Application de la correction à toute la série
    mod_quantiles = cdf_model(mod_series.values)
    corrected_values = inv_cdf_obs(mod_quantiles)
    
    # Création de la série de sortie
    return pd.Series(
        corrected_values, 
        index=mod_series.index, 
        name=f"{mod_series.name}_corrected"
    )

In [ ]:
def convertir_q_en_rh(q_kgkg, temperature_C, pression_hPa=1013.25):
    """
    Convertit une série d'humidité spécifique (kg/kg) et de température (°C)
    en humidité relative (%) en supposant une pression constante.

    Paramètres :
        q_kgkg : pd.Series ou np.array d'humidité spécifique (kg/kg)
        temperature_C : pd.Series ou np.array de température (°C)
        pression_hPa : pression atmosphérique en hPa (par défaut = 1013.25)

    Retour :
        pd.Series ou np.array d'humidité relative (%) — même type que l'entrée
    """
    q = np.asarray(q_kgkg)
    T = np.asarray(temperature_C)

    # Pression partielle de vapeur d'eau (e) [hPa]
    e = (q * pression_hPa) / (0.622 + 0.378 * q)

    # Pression de vapeur saturante (e_s) [hPa] — formule de Tetens
    e_s = 6.112 * np.exp((17.67 * T) / (T + 243.5))

    # Humidité relative RH [%]
    RH = 100 * e / e_s
    RH = np.clip(RH, 0, 100)

    # Renvoyer dans le même format que l'entrée
    if isinstance(q_kgkg, pd.Series):
        return pd.Series(RH, index=q_kgkg.index, name='RH_%')
    else:
        return RH


In [ ]:
def build_and_train_linear(
    X,
    y,
    années_entrainement=('2018', '2019'),
    epochs=100,
    batch_size=8
):
    """
    Entraîne un réseau de neurones pour prédire une variable (ex. température)
    à partir d'une série temporelle d'entrée, toutes deux au format Series.

    Paramètres :
    -----------
    X : pd.Series
        Série d'entrée (ex : température issue de DRIAS corrigé).
    y : pd.Series
        Série cible à prédire (ex : température mesurée à la basilique).
    années_entrainement : tuple(str, str)
        Période d'entraînement (ex. ('2018', '2019')).
    epochs : int
        Nombre d’époques d'entraînement.
    batch_size : int
        Taille des batchs.

    Retourne :
    ----------
    model : modèle Keras entraîné
    df_prédictions : pd.Series des prédictions pour toutes les dates de X
    history : historique d'entraînement du modèle
    """

    # Vérification des types
    if not isinstance(X, pd.Series) or not isinstance(y, pd.Series):
        raise ValueError("X et y doivent être des pd.Series.")

    # 1. Restriction aux années d'entraînement
    X_train = X.loc[années_entrainement[0]:années_entrainement[1]]
    y_train = y.loc[années_entrainement[0]:années_entrainement[1]]

    # 2. Aligner les index
    X_train, y_train = X_train.align(y_train, join='inner')

    # 3. Mise en forme pour Keras
    X_train_vals = X_train.values.reshape(-1, 1)
    y_train_vals = y_train.values.reshape(-1, 1)

    # 4. Définition du modèle
    model = Sequential([
        Dense(32, activation='relu', input_shape=(1,)),
        Dense(16, activation='relu'),
        Dense(1)
    ])

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])

    # 5. Entraînement
    history = model.fit(
        X_train_vals, y_train_vals,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.2,
        verbose=1
    )

    # 6. Prédictions sur toutes les dates de X
    X_all_vals = X.values.reshape(-1, 1)
    y_pred = model.predict(X_all_vals).flatten()
    df_prédictions = pd.Series(y_pred, index=X.index)

    return model, df_prédictions, history


In [ ]:
def plot_each_year_separately(serie1, serie2, label1='Série 1', label2='Série 2'):
    """
    Affiche une figure distincte pour chaque année, en comparant deux séries de température 
    sur les dates communes.

    Paramètres :
    -----------
    serie1, serie2 : pd.Series
        Séries temporelles avec un index datetime.
    label1, label2 : str
        Noms à afficher dans les légendes.
    """

    # Aligner sur les dates communes
    serie1, serie2 = serie1.align(serie2, join='inner')

    # Extraire les années communes
    années = sorted(set(serie1.index.year))

    for annee in années:
        s1 = serie1[serie1.index.year == annee].copy()
        s2 = serie2[serie2.index.year == annee].copy()

        if s1.empty or s2.empty:
            continue

        # Créer une figure par année
        plt.figure(figsize=(12, 4))
        plt.plot(s1.index, s1.values, label=label1, color='blue')
        plt.plot(s2.index, s2.values, label=label2, color='orange')
        plt.title(f"Évolution de la température – {annee}")
        plt.xlabel({annee})
        plt.ylabel("Température (°C)")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

def train_temperature_model(X, y, epochs=100, batch_size=32, validation_split=0.2):
    """
    Construit, compile et entraîne un modèle Keras pour prédire 24 températures horaires.

    Paramètres
    ----------
    X : ndarray shape (n_samples, n_features)
        Tableau des caractéristiques d’entrée (ex. Tmin, Tmoy, Tmax, jour_annee, mois).
    y : ndarray shape (n_samples, n_outputs)
        Tableau des cibles (24 températures horaires).
    epochs : int, optional (default=100)
        Nombre d’époques d’entraînement.
    batch_size : int, optional (default=32)
        Taille du batch.
    validation_split : float, optional (default=0.2)
        Fraction des données réservée à la validation.

    Retour
    ------
    model : keras.Model
        Le modèle entraîné.
    history : keras.callbacks.History
        L’historique d’entraînement (pertes et métriques).
    """
    input_dim  = X.shape[1]
    output_dim = y.shape[1]

    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(output_dim)                         # régression, donc pas d’activation finale
    ])

    model.compile(optimizer=Adam(),
                  loss='mse',
                  metrics=['mae'])

    history = model.fit(X, y,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_split=validation_split,
                        verbose=1)

    return model, history


#### **Dataset DRIAS RCP**

In [ ]:
data_rcp_2_6 = load_data_drias('Drias/CNRM-CERFACS-CNRM-CM5_CNRM-ALADIN63_rcp2.6_METEO-FRANCE_ADAMONT-France_SAFRAN_day_2006_2100.xlsx')

In [ ]:
data_rcp_4_5 = load_data_drias('Drias/CNRM-CERFACS-CNRM-CM5_CNRM-ALADIN63_rcp4.5_METEO-FRANCE_ADAMONT-France_SAFRAN_day_2006_2100.xlsx')

In [ ]:
data_rcp_8_5 = load_data_drias('Drias/CNRM-CERFACS-CNRM-CM5_CNRM-ALADIN63_rcp8.5_METEO-FRANCE_ADAMONT-France_SAFRAN_day_2006_2100.xlsx')

#### **Dataset Historique-Météo** 

In [ ]:
# Liste des années à importer
annees = range(2009, 2026)

# Liste pour stocker les DataFrames
dfs = []

# Boucle sur les années
for annee in annees:
    nom_fichier = f"Historique-Météo/export-reims{annee}.csv"
    df = pd.read_csv(nom_fichier, skiprows=3)
    dfs.append(df)

# Concatène tous les DataFrames en un seul
data_climate_reims_2009_2025 = pd.concat(dfs, ignore_index=True)

In [ ]:
data_climate_reims_2009_2025['DATE'] = pd.to_datetime(data_climate_reims_2009_2025['DATE'], format='%Y-%m-%d')
data_climate_reims_2009_2025.set_index('DATE', inplace=True)
data_climate_reims_2009_2025.head()

In [ ]:
# MEAN_TEMPERATURE_C
colonnes_temp_horaires = [
    'TEMPERATURE_MORNING_C_6H',
    'TEMPERATURE_NOON_C_12H',
    'TEMPERATURE_EVENING_C_18H',
    'TEMPERATURE_NIGHT_C_3H',
    'TEMPERATURE_9H',
    'TEMPERATURE_15H',
    'TEMPERATURE_21H',
    'TEMPERATURE_MIDNIGHT_0H'
]

# Calcul de la moyenne journalière basée uniquement sur les températures horaires
data_climate_reims_2009_2025['MEAN_TEMPERATURE_C'] = data_climate_reims_2009_2025[colonnes_temp_horaires].mean(axis=1)
data_climate_reims_2009_2025['MEAN_TEMPERATURE_C']

In [ ]:
# récupérer les index communs data rcp2.6
common_index_data_rcp_2_6 = data_rcp_2_6.index.intersection(data_climate_reims_2009_2025.index)
data_rcp_2_6_2009_2025 = data_rcp_2_6.loc[common_index_data_rcp_2_6]

In [ ]:
# récupérer les index communs data rcp4.5
common_index_data_rcp_4_5 = data_rcp_4_5.index.intersection(data_climate_reims_2009_2025.index)
data_rcp_4_5_2009_2025 = data_rcp_4_5.loc[common_index_data_rcp_4_5]

In [ ]:
# récupérer les index communs data rcp8.5
common_index_data_rcp_8_5 = data_rcp_8_5.index.intersection(data_climate_reims_2009_2025.index)
data_rcp_8_5_2009_2025 = data_rcp_8_5.loc[common_index_data_rcp_8_5]

#### **Dataset Prunay**

##### **🌦️ Description des colonnes - Station météo Reims - Prunay**

signification de chaque variable issue des données météorologiques :

| **Nom de la colonne**             | **Description**                                                                 | **Unité**                   |
|----------------------------------|----------------------------------------------------------------------------------|-----------------------------|
| `time`                           | Horodatage des mesures                                                           | Date/Heure (ISO 8601)       |
| `10m-U_wind_ms`                  | Composante est-ouest du vent à 10 mètres                                        | m/s                         |
| `10m-V_wind_ms`                  | Composante nord-sud du vent à 10 mètres                                         | m/s                         |
| `2m_DewPoint_Temperature_K`      | Température du point de rosée à 2 mètres                                        | Kelvin (K)                  |
| `2m_Temperature_K`               | Température de l'air à 2 mètres                                                 | Kelvin (K)                  |
| `Surface_Pressure_Pa`           | Pression atmosphérique au niveau de la surface                                  | Pascals (Pa)                |
| `RH`                             | Humidité relative de l'air                                                       | Pourcentage (%)             |
| `Wind_Speed`                     | Vitesse du vent calculée à partir des composantes U et V                        | m/s                         |
| `Wind_Direction_CMEMS`           | Direction du vent (d'origine) selon les données CMEMS                           | Degrés (°)                  |
| `Wind_Direction_Charbel`         | Direction du vent selon un calcul propre (ex. méthode "Charbel")                | Degrés (°)                  |
| `Wind_Direction_Era5`            | Direction du vent selon les données ERA5 (réanalyse ECMWF)                      | Degrés (°)                  |



In [ ]:
data_prunay = pd.read_csv('Atmo\donnees_Celine_LJ.csv')
data_prunay.head()

In [ ]:
data_prunay.dtypes

In [ ]:
data_prunay['time'] = pd.to_datetime(data_prunay['time'], format='%Y-%m-%d %H:%M:%S')
data_prunay.set_index('time', inplace=True)
data_prunay.head()

In [ ]:
# récupérer les index communs data rcp2.6
common_index_data_rcp_2_6 = data_rcp_2_6.index.intersection(data_prunay.index)
data_rcp_2_6_ = data_rcp_2_6.loc[common_index_data_rcp_2_6]

In [ ]:
# récupérer les index communs data rcp4.5
common_index_data_rcp_4_5 = data_rcp_4_5.index.intersection(data_prunay.index)
data_rcp_4_5_ = data_rcp_4_5.loc[common_index_data_rcp_4_5]

In [ ]:
# récupérer les index communs data rcp8.5
common_index_data_rcp_8_5 = data_rcp_8_5.index.intersection(data_prunay.index)
data_rcp_8_5_ = data_rcp_8_5.loc[common_index_data_rcp_8_5]

In [ ]:
data_prunay.drop(['latitude', 'longitude'], axis=1, inplace=True)

##### Correlation Barplot with meantemp feature

In [ ]:
plt.figure(figsize=(8, 4))
sns.set_style('darkgrid')

correlation_matrix = round(data_prunay.corr(), 2)

correlation_with_trgt = correlation_matrix['2m_Temperature_K'].sort_values(ascending=False)

ax = sns.barplot(x=correlation_with_trgt.index, y=correlation_with_trgt, palette='viridis')

plt.title('Correlation with meantemp', size= 20)
plt.xlabel('Features')
plt.ylabel('Correlation')

for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
data_prunay = data_prunay[["2m_Temperature_K", "RH"]].copy()

In [ ]:
data_prunay['2m_Temperature_C'] = data_prunay['2m_Temperature_K'] - 273.15
data_prunay.drop(['2m_Temperature_K'], axis=1, inplace=True)

In [ ]:
data_prunay = data_prunay.resample('D').mean()

In [ ]:
data_prunay.shape

In [ ]:
dl_train_data_prunay, dl_val_data_prunay, dl_test_data_prunay = split_dataframe(data_prunay)

In [ ]:
dl_train_data_prunay

In [ ]:
minmax_scaler_data_prunay = MinMaxScaler()  # scaler for humidity
target_transformer_data_prunay = MinMaxScaler()   # scaler for target (meantemp)

In [ ]:
dl_train_data_prunay['2m_Temperature_C'] = target_transformer_data_prunay.fit_transform(dl_train_data_prunay[['2m_Temperature_C']]) # target
dl_train_data_prunay['RH'] = minmax_scaler_data_prunay.fit_transform(dl_train_data_prunay[['RH']]) # minmax for humidity

dl_val_data_prunay['2m_Temperature_C'] = target_transformer_data_prunay.transform(dl_val_data_prunay[['2m_Temperature_C']])
dl_val_data_prunay['RH'] = minmax_scaler_data_prunay.transform(dl_val_data_prunay[['RH']])

dl_test_data_prunay['2m_Temperature_C'] = target_transformer_data_prunay.transform(dl_test_data_prunay[['2m_Temperature_C']])
dl_test_data_prunay['RH'] = minmax_scaler_data_prunay.transform(dl_test_data_prunay[['RH']])

In [ ]:
# Create sequences
sequence_length = 3 # Example sequence length (adjust based on your data and experimentation)
X_data_prunay_train, y_data_prunay_train = create_dataset(dl_train_data_prunay, dl_train_data_prunay['2m_Temperature_C'], sequence_length)
X_data_prunay_val, y_data_prunay_val = create_dataset(dl_val_data_prunay, dl_val_data_prunay['2m_Temperature_C'], sequence_length)
X_data_prunay_test, y_data_prunay_test = create_dataset(dl_test_data_prunay, dl_test_data_prunay['2m_Temperature_C'], sequence_length)

In [ ]:
model_data_prunay, history_data_prunay, val_loss_data_prunay = build_and_train_bilstm(X_data_prunay_train, y_data_prunay_train, X_data_prunay_val, y_data_prunay_val, sequence_length , units=100, epochs=50, batch_size=8)

In [ ]:
model_data_prunay.summary()

In [ ]:
# Get training and validation losses from history
training_loss = history_data_prunay.history['loss']
validation_loss = history_data_prunay.history['val_loss']

# Plot loss values over epochs
plt.plot(training_loss, label='Training Loss')
plt.plot(validation_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
# Make predictions data train
y_pred_inv, y_val_inv = predict_and_inverse_transform(model_data_prunay, X_data_prunay_val, y_data_prunay_val, target_transformer_data_prunay)

In [ ]:
# Eavluate Model data train 
evaluate_model(y_val_inv, y_pred_inv)

In [ ]:
data_prunay

In [ ]:
plt.figure(figsize=(14, 7))
start = len(y_data_prunay_train) + sequence_length
end = start + len(y_data_prunay_val)

plt.plot(data_prunay.index[start:end], y_val_inv, label='True Values')
plt.plot(data_prunay.index[start:end], y_pred_inv, label='Predictions', linestyle='dashed')
plt.xlabel('Date')
plt.ylabel('Mean Temperature')
plt.title('Mean Temperature Predictions vs True Values (Data Validation)')
plt.legend()
plt.show()

In [ ]:
# Make predictions data test
y_pred_inv_test, y_val_inv_test = predict_and_inverse_transform(model_data_prunay, X_data_prunay_test, y_data_prunay_test, target_transformer_data_prunay)

In [ ]:
# Eavluate Model data train 
evaluate_model(y_val_inv_test, y_pred_inv_test)

In [ ]:
# Calcul des bornes d'index pour la partie test
start_test = len(y_data_prunay_train) + len(y_data_prunay_val) + sequence_length
end_test = start_test + len(y_data_prunay_test)

# Tracé
plt.figure(figsize=(14, 7))
plt.plot(data_prunay.index[start_test:end_test], y_val_inv_test, label='True Values')
plt.plot(data_prunay.index[start_test:end_test], y_pred_inv_test, label='Predictions', linestyle='dashed')
plt.xlabel('Date')
plt.ylabel('Mean Temperature')
plt.title('Mean Temperature Predictions vs True Values (Data Test)')
plt.legend()
plt.show()

In [ ]:
data_prunay

#### **Dataset Releve iButton Basilique Saint-Rémi**

In [ ]:
data_basilique_2018_2019 = pd.read_excel('Releve iButton Basilique Saint Rémi/DATA_processing_iButton_2018_2019.xlsx',  index_col='Date Heure', parse_dates=True)
data_basilique_2018_2019.head()

In [ ]:
plt.figure(figsize=(25,5))
labels = data_basilique_2018_2019[Temp_HR].isnull().sum().sort_values(ascending=False).index.to_list()
sizes = data_basilique_2018_2019[Temp_HR].isnull().sum().sort_values(ascending=False).to_list()
sns.barplot(x=labels, y=sizes)
plt.grid()
plt.ylim((0.0, float(data_basilique_2018_2019.shape[0])))
plt.show()

In [ ]:
data_bsr_2018_2019 = data_basilique_2018_2019.resample('D').mean().copy()
data_bsr_2018_2019 = data_bsr_2018_2019[["S2STemp", "S2SHR"]]
data_bsr_2018_2019 = data_bsr_2018_2019.interpolate(method='linear') ## Pour remplacer les valeurs NAN DU 2018-06-10, 2018-06-11, 2018-06-12, 2018-06-13
data_bsr_2018_2019

In [ ]:
dl_train_data_bsr, dl_val_data_bsr, dl_test_data_bsr = split_dataframe(data_bsr_2018_2019)

In [ ]:
minmax_scaler_data_bsr = MinMaxScaler()  # scaler for humidity
target_transformer_data_bsr = MinMaxScaler()   # scaler for target (meantemp)

In [ ]:
dl_train_data_bsr['S2STemp'] = target_transformer_data_bsr.fit_transform(dl_train_data_bsr[['S2STemp']]) # target
dl_train_data_bsr['S2SHR'] = minmax_scaler_data_bsr.fit_transform(dl_train_data_bsr[['S2SHR']]) # minmax for humidity

dl_val_data_bsr['S2STemp'] = target_transformer_data_bsr.transform(dl_val_data_bsr[['S2STemp']])
dl_val_data_bsr['S2SHR'] = minmax_scaler_data_bsr.transform(dl_val_data_bsr[['S2SHR']])

dl_test_data_bsr['S2STemp'] = target_transformer_data_bsr.transform(dl_test_data_bsr[['S2STemp']])
dl_test_data_bsr['S2SHR'] = minmax_scaler_data_bsr.transform(dl_test_data_bsr[['S2SHR']])

In [ ]:
# Create sequences
sequence_length = 3  # Example sequence length (adjust based on your data and experimentation)
X_data_bsr_train, y_data_bsr_train = create_dataset(dl_train_data_bsr, dl_train_data_bsr['S2STemp'], sequence_length)
X_data_bsr_val, y_data_bsr_val = create_dataset(dl_val_data_bsr, dl_val_data_bsr['S2STemp'], sequence_length)
X_data_bsr_test, y_data_bsr_test = create_dataset(dl_test_data_bsr, dl_test_data_bsr['S2STemp'], sequence_length)

In [ ]:
model_data_bsr, history_data_bsr, val_loss_data_bsr = build_and_train_bilstm(X_data_bsr_train, y_data_bsr_train, X_data_bsr_val, y_data_bsr_val, sequence_length , units=100, epochs=30, batch_size=1)

In [ ]:
model_data_bsr.summary()

In [ ]:
# Get training and validation losses from history
training_loss = history_data_bsr.history['loss']
validation_loss = history_data_bsr.history['val_loss']

# Plot loss values over epochs
plt.plot(training_loss, label='Training Loss')
plt.plot(validation_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
# Make predictions data train
y_pred_data_bsr_inv, y_val_data_bsr_inv = predict_and_inverse_transform(model_data_bsr, X_data_bsr_val, y_data_bsr_val, target_transformer_data_bsr)

In [ ]:
# Eavluate Model data train 
evaluate_model(y_val_data_bsr_inv, y_pred_data_bsr_inv)

In [ ]:
# Calcul de l'index de début et de fin pour la validation
start_val = len(y_data_bsr_train) + sequence_length
end_val = start_val + len(y_data_bsr_val)

# Tracé
plt.figure(figsize=(14, 7))
plt.plot(data_bsr_2018_2019.index[start_val:end_val], y_val_data_bsr_inv, label='True Values')
plt.plot(data_bsr_2018_2019.index[start_val:end_val], y_pred_data_bsr_inv, label='Predictions', linestyle='dashed')
plt.xlabel('Date')
plt.ylabel('Mean Temperature')
plt.title('Mean Temperature Predictions vs True Values (Data Validation)')
plt.legend()
plt.show()


In [ ]:
# Make predictions data test
y_pred_inv_data_bsr_test, y_test_inv_data_bsr_test = predict_and_inverse_transform(model_data_bsr, X_data_bsr_test, y_data_bsr_test, target_transformer_data_bsr)

In [ ]:
# Eavluate Model data test
evaluate_model(y_test_inv_data_bsr_test, y_pred_inv_data_bsr_test)

In [ ]:
# Calcul des bornes d'index pour la partie test
start_test = len(y_data_bsr_train) + len(y_data_bsr_val) + sequence_length
end_test = start_test + len(y_data_bsr_test)

# Tracé
plt.figure(figsize=(14, 7))
plt.plot(data_bsr_2018_2019.index[start_test:end_test], y_test_inv_data_bsr_test, label='True Values')
plt.plot(data_bsr_2018_2019.index[start_test:end_test], y_pred_inv_data_bsr_test, label='Predictions', linestyle='dashed')
plt.xlabel('Date')
plt.ylabel('Mean Temperature')
plt.title('Mean Temperature Predictions vs True Values (Data Test)')
plt.legend()
plt.show()


#### **Correction des biais des données du DRIAS**

In [ ]:
temp_mean_max_min(data_rcp_4_5_, data_prunay, "Data DRIAS RCP 2.6", "Data Station Prunay")

In [ ]:
data_prunay

In [ ]:
# Comparaison des performances entre les RCP
rcp_compare = {
    'RCP 2.6': evaluate_model(data_prunay["2m_Temperature_C"], data_rcp_2_6_["tasAdjust"]),
    'RCP 4.5': evaluate_model(data_prunay["2m_Temperature_C"], data_rcp_4_5_["tasAdjust"]),
    'RCP 8.5': evaluate_model(data_prunay["2m_Temperature_C"], data_rcp_8_5_["tasAdjust"])
}

# Mettre dans un DataFrame avec MAE, RMSE, R2 en index
rcp_compare = pd.DataFrame(rcp_compare, index=['MAE', 'RMSE', 'R2'])

print()
print()
# Affichage clair
print(rcp_compare.T.round(3))

In [ ]:
tasAdjust_corrected = quantile_mapping_correction(data_prunay["2m_Temperature_C"], data_rcp_2_6["tasAdjust"])
data_rcp_2_6["tasAdjust"] = tasAdjust_corrected

In [ ]:
# récupérer les index communs data rcp2.6
common_index_data_rcp_2_6 = data_rcp_2_6.index.intersection(data_prunay.index)
data_rcp_2_6_ = data_rcp_2_6.loc[common_index_data_rcp_2_6]

In [ ]:
# Comparaison des performances entre les RCP
rcp_compare = {
    'RCP 2.6': evaluate_model(data_prunay["2m_Temperature_C"], data_rcp_2_6_["tasAdjust"]),
}

# Mettre dans un DataFrame avec MAE, RMSE, R2 en index
rcp_compare = pd.DataFrame(rcp_compare, index=['MAE', 'RMSE', 'R2'])

print()
print()
# Affichage clair
print(rcp_compare.T.round(3))

In [ ]:
temp_mean_max_min(data_rcp_4_5_, data_prunay, "Data DRIAS RCP 2.6", "Data Station Prunay")

In [ ]:
RH = convertir_q_en_rh(data_rcp_2_6["hussAdjust"], data_rcp_2_6["tasAdjust"])
data_rcp_2_6["RH"] = RH


In [ ]:
data_rcp_2_6

In [ ]:
# récupérer les index communs data rcp2.6
common_index_data_rcp_2_6 = data_rcp_2_6.index.intersection(data_prunay.index)
data_rcp_2_6_ = data_rcp_2_6.loc[common_index_data_rcp_2_6]

In [ ]:
# Comparaison des performances entre les RCP
RH_compare = {
    'RH': evaluate_model(data_prunay["RH"], data_rcp_2_6_["RH"]),
}

# Mettre dans un DataFrame avec MAE, RMSE, R2 en index
rcp_compare = pd.DataFrame(RH_compare, index=['MAE', 'RMSE', 'R2'])

print()
print()
# Affichage clair
print(rcp_compare.T.round(3))

In [ ]:
RH_corrected = quantile_mapping_correction(data_prunay["RH"], data_rcp_2_6["RH"])
data_rcp_2_6["RH"] = RH_corrected

In [ ]:
# récupérer les index communs data rcp2.6
common_index_data_rcp_2_6 = data_rcp_2_6.index.intersection(data_prunay.index)
data_rcp_2_6_ = data_rcp_2_6.loc[common_index_data_rcp_2_6]

In [ ]:
# Comparaison des performances entre les RCP
RH_compare = {
    'RH': evaluate_model(data_prunay["RH"], data_rcp_2_6_["RH"]),
}

# Mettre dans un DataFrame avec MAE, RMSE, R2 en index
rcp_compare = pd.DataFrame(RH_compare, index=['MAE', 'RMSE', 'R2'])

print()
print()
# Affichage clair
print(rcp_compare.T.round(3))

In [ ]:
data_rcp_2_6 = data_rcp_2_6[["RH", "tasAdjust"]].copy()

In [ ]:
data_rcp_2_6 = data_rcp_2_6.rename(columns={'tasAdjust': '2m_Temperature_C'})

In [ ]:
data_rcp_2_6['2m_Temperature_C'] = target_transformer_data_prunay.transform(data_rcp_2_6[['2m_Temperature_C']])
data_rcp_2_6['RH'] = minmax_scaler_data_prunay.transform(data_rcp_2_6[['RH']])

In [ ]:
def create_dataset(X, time_steps=1): 
    Xs = []   
    for i in range(len(X) - time_steps):   
        v = X.iloc[i:(i + time_steps)].values 
        Xs.append(v)      
    return np.array(Xs)

In [ ]:
data_rcp_2_6

In [ ]:
# Create sequences
sequence_length = 3 # Example sequence length (adjust based on your data and experimentation)
X = create_dataset(data_rcp_2_6, sequence_length)

In [ ]:
X

In [ ]:
# Prédiction
y_pred = model_data_prunay.predict(X)

# Inversion de la transformation des prédictions
y_pred_inv = target_transformer_data_prunay.inverse_transform(y_pred)

In [ ]:
y_pred_inv

In [ ]:
# sequence_length
start_idx = sequence_length
end_idx = start_idx + len(y_pred_inv)

# Tracé
plt.figure(figsize=(30, 7))
plt.plot(data_rcp_2_6.index[start_idx:end_idx], y_pred_inv, label='Predictions', linestyle='dashed')
plt.xlabel('Date')
plt.ylabel('Mean Temperature')
plt.title('Mean Temperature Predictions (RCP 2.6)')
plt.legend()
plt.show()

In [ ]:
# sequence_length
start_idx = sequence_length
end_idx = start_idx + len(y_pred_inv)

y_pred_flat = y_pred_inv.ravel()  

# Créer le DataFrame
df_prediction_rcp_2_6_corrige = pd.DataFrame({
    'date': data_rcp_2_6.index[start_idx:end_idx],
    'prediction': y_pred_flat
})

In [ ]:
df_prediction_rcp_2_6_corrige.tail()

In [ ]:
df_prediction_rcp_2_6_corrige.set_index('date', inplace=True)

In [ ]:
# récupérer les index communs data rcp2.6
common_index_data_rcp_2_6 = df_prediction_rcp_2_6_corrige.index.intersection(data_prunay.index)
data_rcp_2_6_ = df_prediction_rcp_2_6_corrige.loc[common_index_data_rcp_2_6]

In [ ]:
data_rcp_2_6_

In [ ]:
data_prunay["2m_Temperature_C"][common_index_data_rcp_2_6]

In [ ]:
# Comparaison des performances entre les RCP
rcp_compare = {
    'RCP 2.6': evaluate_model(data_prunay["2m_Temperature_C"][common_index_data_rcp_2_6], data_rcp_2_6_["prediction"]),
}

# Mettre dans un DataFrame avec MAE, RMSE, R2 en index
rcp_compare = pd.DataFrame(rcp_compare, index=['MAE', 'RMSE', 'R2'])

print()
print()
# Affichage clair
print(rcp_compare.T.round(3))

In [ ]:
data_rcp_2_6_

In [ ]:
data_prunay

In [ ]:
data_rcp_2_6_ = data_rcp_2_6.rename(columns={'prediction': 'tasAdjust'})

In [ ]:
data_prunay = data_prunay.rename(columns={'tasAdjust': '2m_Temperature_C'})

In [ ]:
temp_mean_max_min(data_rcp_4_5_, data_prunay, "Data DRIAS RCP 2.6 Corrigé", "Data Station Prunay")

In [ ]:
data_prunay

In [ ]:
df_prediction_rcp_2_6_corrige

In [ ]:
dl_train_data_bsr_linear, dl_val_data_bsr_linear, dl_test_data_prunay_bsr_linear = split_dataframe(data_prunay, 0.8, 0.2)

In [ ]:
linear_regression_model, linear_regression_prédictions, linear_regression_history  = build_and_train_linear(
    X = df_prediction_rcp_2_6_corrige["prediction"],
    y = data_bsr_2018_2019["S2STemp"],
    années_entrainement = ('2018', '2019'),
    epochs = 50,
    batch_size = 32
)

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(linear_regression_history.history['loss'], label='Loss')
plt.plot(linear_regression_history.history['val_loss'], label='Validation Loss')
plt.title("Courbe de l'erreur pendant l'entraînement")
plt.xlabel("Epoch")
plt.ylabel("MSE")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Plot the results Data Validation
plt.figure(figsize=(14, 7))
plt.plot(data_bsr_2018_2019.index[len(y_data_bsr_train): len(y_data_bsr_train) + len(y_data_bsr_val)], y_val_data_bsr_inv, label='True Values')
plt.plot(data_bsr_2018_2019.index[len(y_data_bsr_train): len(y_data_bsr_train) + len(y_data_bsr_val)], y_pred_data_bsr_inv, label='Predictions', linestyle='dashed')
plt.xlabel('Date')
plt.ylabel('Mean Temperature')
plt.title('Mean Temperature Predictions vs True Values (Data Validation)')
plt.legend()
plt.show()

In [ ]:
linear_regression_prédictions = df = pd.DataFrame({'2m_Temperature_C': linear_regression_prédictions})

In [ ]:
temp_mean_max_min(data_rcp_4_5_, linear_regression_prédictions, "Data DRIAS RCP 2.6 Corrigé", "Saint-Remy")

In [ ]:
linear_regression_prédictions

In [ ]:
plot_each_year_separately(data_prunay["2m_Temperature_C"], data_climate_reims_2009_2025["MEAN_TEMPERATURE_C"], "Station Prunay", "Historique-Météo")

In [ ]:
data_prunay

In [ ]:
data_climate_reims_2009_2025

In [ ]:
data_climate_reims_2009_2025.columns

In [ ]:
# récupérer les index communs data rcp2.6
common_index_data_prunay_historique_meteo = data_climate_reims_2009_2025.index.intersection(data_prunay.index)
data_climate_reims_2009_2025_ = data_climate_reims_2009_2025.loc[common_index_data_prunay_historique_meteo]
data_prunay_ = data_prunay.loc[common_index_data_prunay_historique_meteo]

In [ ]:
evaluate_model(data_prunay_["2m_Temperature_C"], data_climate_reims_2009_2025_["MEAN_TEMPERATURE_C"]),

In [ ]:
data_prunay = pd.read_csv('Atmo\donnees_Celine_LJ.csv')
data_prunay['time'] = pd.to_datetime(data_prunay['time'], format='%Y-%m-%d %H:%M:%S')
data_prunay.set_index('time', inplace=True)
data_prunay['2m_Temperature_C'] = data_prunay['2m_Temperature_K'] - 273.15

# Détermine le nombre de lignes nécessaires pour les subplots
rows = len(tour_sud_2_temp)
cols = 2  # Deux colonnes : une pour data_1, une pour data_2

# Copie des bases de données pour éviter de modifier les originales
data_copy_1 = data_basilique_2018_2019.copy()
data_copy_2 = data_prunay.copy()

# Suppression des doublons pour éviter des erreurs d'affichage ou d'analyse
data_copy_1 = data_copy_1[~data_copy_1.index.duplicated()]

# Filtrage des données pour ne conserver que les années 2018 et 2019
data_copy_1 = data_copy_1[data_copy_1.index.year.isin([2018, 2019])]

i = 1  # Compteur pour positionner les subplots

# Définition de la taille de la figure pour un affichage correct des graphiques
plt.figure(figsize=(25, 4 * rows))

for column in tour_sud_2_temp:

    # Tracé du premier subplot : données brutes de data_1
    plt.subplot(rows, cols, i)
    plt.scatter(data_copy_1[column].index, data_copy_1[column].values, c="grey")
    plt.ylim(-10, 50)
    plt.title(f"Original Data {designations[column]}: {column}")
    plt.grid()

    # Tracé du deuxième subplot : données de la station Prunay
    plt.subplot(rows, cols, i+1)
    plt.scatter(data_copy_2.loc[data_copy_2.index.year.isin([2018, 2019]), "2m_Temperature_C"].index, data_copy_2.loc[data_copy_2.index.year.isin([2018, 2019]), "2m_Temperature_C"].values, c="grey")
    # plt.ylim(data_copy_1[column].min()-5.0, data_copy_1[column].max()+5.0)
    plt.title("Station Prunay")
    plt.grid()

    # Incrémentation du compteur pour passer au subplot suivant
    i += 2

# Ajustement automatique de l'affichage des graphiques pour éviter les chevauchements
plt.tight_layout()
plt.show()

In [ ]:
data_prunay

In [ ]:
data_prunay